## Imports

In [1]:
#standard useful libraries
import pandas as pd
import numpy as np
import random

#model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#neural network dependencies
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

C:\Users\Amy Perez\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading Data

In [2]:
#Loading data from csv
pipes_df = pd.read_csv("Pipe_data.csv")
pipes_df.head()

C:\Users\Amy Perez\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (57,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FID,MOID,DIAMETER,PIPE_LENGTH,MATERIAL,DESCRIPTION,YEAR_INSTALLED,PRESSURE_ZONE,LINING,LINE_YEAR,...,WORK_ORDER_NUMBER,Comments,ReportDate,StartDateT,CompletedD,ReportedY,SrvcRqst_1,MATERIAL_1,MUSYM,FAILURE
0,16420,P35881,8.0,861.090481,M.J.,Distribution,1952,Hemp 1175,,0,...,1143689,PER UNIT 819\n\rREQUEST 1 6X12 PLATE AND 4 BAG...,2014-10-25,2014-11-16,2014-11-16,2014,Water Request,CAS,Ub,1
1,44465,P88079,8.0,749.511184,TYT,Distribution,1965,Hemp 1175,,0,...,1311148,MAIN BREAK,2015-08-18,2015-08-18,2015-08-18,2015,Water Request,DIP,ReD,1
2,23810,P53764,8.0,171.554040,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
3,52630,P400741,8.0,42.761097,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
4,10510,P20210,12.0,31.300481,DIP,Transmission,2006,Hemp 1175,,0,...,1628288,PER J. SKINNER MAIN BREAK AT LOCATION\nCall ta...,2017-01-22,2017-02-28,2017-02-28,2017,Turn off burst pipe,DIP,Ub,1


In [3]:
#get dummies for columns where applicable--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['MATERIAL'], prefix='MATERIAL')], axis=1)
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['DESCRIPTION'], prefix='DESCRIPTION')], axis=1); 
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['PRESSURE_ZONE'], prefix='P-ZONE')], axis=1);

#drop original columns--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df2 = pipes_df.drop(columns={ 'MATERIAL',
 'DESCRIPTION',
 'PRESSURE_ZONE',})

#Condense correlating PATTERN + DEMAND columns into 1 column
pipes_df2 = pipes_df2.rename(columns={"DEMAND1":"Demand_RES", "DEMAND2":"Demand_NON-RES","DEMAND3":"Demand_WHOLE","DEMAND4":"Demand_FLUSH","DEMAND5":"Demand_UFW",
                                     "DEMAND6":"Demand_RES-DISTR","DEMAND7":"Demand_NON RES-DISTR","DEMAND8":"Demand_2030-CWTP-NS"})

#Delete unneeded columns
#NOTE to Team: I didn’t see any data at all in the ‘DEMAND9’, ‘PATTERN9’, ‘DEMAND10’, ‘PATTERN10’ columns,so I deleted all of these columns
pipes_df3 = pipes_df2.drop(columns={"PATTERN1", "PATTERN2", "PATTERN3", "PATTERN4", "PATTERN5", "PATTERN6", "PATTERN7", "PATTERN8", "PATTERN9", "PATTERN10", "DEMAND9", "DEMAND10"})

In [4]:
#Create new Age column
#Age is being calculated as 2018 - installation year if no report was made
#Report year - installation year otherwise
tempAgeList = []
for row in np.arange(len(pipes_df3['FID'])):
    try:
        value = int(pipes_df3.iloc[row]['ReportDate'][:4]) - pipes_df3.iloc[row]['YEAR_INSTALLED']
    except:
        value = 2018 - pipes_df3.iloc[row]['YEAR_INSTALLED']
    tempAgeList.append(value)
pipes_df3['Age']=tempAgeList

In [6]:
#Add a column to be opposite of FAILURE column for use in neural network training
tempSuccessList = []
for row in np.arange(len(pipes_df3['FID'])):
    value = 1 - pipes_df3.iloc[row]['FAILURE']
    tempSuccessList.append(value)
pipes_df3['SUCCESS'] = tempSuccessList

# Machine Learning

# PreProcessing

In [7]:
#Select which columns to train off of.
#A number of iterations of this were initially used when exploring the data
#In the end it was decided to only use average values for those which have averages

inputColumns = ['DIAMETER', 'PIPE_LENGTH','ROUGHNESS', 'FLOW', 'VELOCITY', 'HEADLOSS',
       'HL1000', 'MAX_FLOW', 'MIN_FLOW', 'AVE_FLOW', 'MAX_VELOCITY',
       'MIN_VELOCITY', 'AVE_VELOCITY', 'MAX_HEADLOSS', 'MIN_HEADLOSS',
       'AVE_HEADLOSS', 'ELEVATION','RUN_ELEV', 'DEMAND',
       'HEAD', 'PRESSURE', 'MAX_HEAD', 'MIN_HEAD', 'AVE_HEAD', 'MAX_PRESS',
       'MIN_PRESS', 'AVE_PRESS', 'DIFF_PRESS','MATERIAL_AMTYT',
       'MATERIAL_CI', 'MATERIAL_CO', 'MATERIAL_COP', 'MATERIAL_CPP',
       'MATERIAL_Copper', 'MATERIAL_DEL', 'MATERIAL_DIP', 'MATERIAL_EARGEO',
       'MATERIAL_GP', 'MATERIAL_HDPE', 'MATERIAL_L.J.', 'MATERIAL_M.J.',
       'MATERIAL_MCW', 'MATERIAL_OTH', 'MATERIAL_PE', 'MATERIAL_PP',
       'MATERIAL_PVC', 'MATERIAL_R.L.J.', 'MATERIAL_RCP', 'MATERIAL_SP',
       'MATERIAL_ST', 'MATERIAL_T.D', 'MATERIAL_T.D.', 'MATERIAL_TTE',
       'MATERIAL_TYT', 'MATERIAL_UNK', 'MATERIAL_W.I.','Age']

In [8]:
#Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pipes_df3[inputColumns],pipes_df3['FAILURE'],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

#Make another split for y which uses the additional 'SUCCESS' column for use in neural network
#Note: Using random state the X_train and X_test would be identical and so need not be kept
_, _, y_train_2, y_test_2 = train_test_split(pipes_df3[inputColumns],pipes_df3[['FAILURE','SUCCESS']],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

In [9]:
#Note: If we were to train and test on the split as it currently is, it will just guess that everything is an unbroken pipe
#This is not desired behavior

#Split X_train into a list of those where the pipes are broken and those which arent
#Then, undersample those pipes which are unbroken to avoid bias
X_train_false = X_train[y_train==0].sample(n=sum(y_train), random_state=42)
X_train_true = X_train[y_train==1].sample(n=sum(y_train), random_state=42)

#Recombine the sampled X_train into one list and keep only those y_train values corresponding to selected indices
X_train_sampled = X_train_true.append(X_train_false)
y_train_sampled = y_train.loc[X_train_sampled.index]
y_train_sampled_2 = y_train_2.loc[X_train_sampled.index]

#Apply min-max scalers to X
#Note: applying min-max too early changed format of dataframe into a np.array and interfered with the sampling process
X_minmax = MinMaxScaler().fit(X_train)
X_train_minmax = X_minmax.transform(X_train_sampled)
X_test_minmax = X_minmax.transform(X_test)

## Logistic Regression

In [10]:
#Set a logistic regression as a model
lrmodel = LogisticRegression()
lrmodel.fit(X_train_minmax,y_train_sampled)
lrmodel.score(X_train_minmax,y_train_sampled)

0.6892797319932998

In [12]:
lrmodel.score(X_test_minmax,y_test)

0.701142585098786

In [13]:
confusion_matrix(y_test,lrmodel.predict(X_test_minmax))

array([[11100,  4680],
       [  342,   682]], dtype=int64)

## Neural Network

In [12]:
#Add a shallow neural network
nnmodel = Sequential()

#Add layers
nnmodel.add(Dense(100, activation='relu', input_dim=X_train_minmax.shape[1]))
nnmodel.add(Dense(2,activation = 'softmax'))

#Compile
nnmodel.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

#Fit the training model to the data
nnmodel.fit(
    X_train_minmax,
    y_train_sampled_2,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 1s - loss: 0.6586 - acc: 0.6074
Epoch 2/100
 - 0s - loss: 0.6410 - acc: 0.6277
Epoch 3/100
 - 0s - loss: 0.6306 - acc: 0.6413
Epoch 4/100
 - 0s - loss: 0.6224 - acc: 0.6489
Epoch 5/100
 - 0s - loss: 0.6120 - acc: 0.6619
Epoch 6/100
 - 0s - loss: 0.6008 - acc: 0.6790
Epoch 7/100
 - 0s - loss: 0.5971 - acc: 0.6786
Epoch 8/100
 - 0s - loss: 0.5882 - acc: 0.6899
Epoch 9/100
 - 0s - loss: 0.5823 - acc: 0.6960
Epoch 10/100
 - 0s - loss: 0.5776 - acc: 0.7018
Epoch 11/100
 - 0s - loss: 0.5699 - acc: 0.7127
Epoch 12/100
 - 0s - loss: 0.5654 - acc: 0.7194
Epoch 13/100
 - 0s - loss: 0.5616 - acc: 0.7131
Epoch 14/100
 - 0s - loss: 0.5571 - acc: 0.7245
Epoch 15/100
 - 0s - loss: 0.5570 - acc: 0.7240
Epoch 16/100
 - 0s - loss: 0.5559 - acc: 0.7255
Epoch 17/100
 - 0s - loss: 0.5500 - acc: 0.7265
Epoch 18/100
 - 0s - loss: 0.5512 - acc: 0.7263
Epoch 19/100
 - 0s - loss: 0.5493 - acc: 0.7240
Epoch 20/100
 - 0s - loss: 0.5450 - acc: 0.7307
Epoch 21/100
 - 0s - loss: 0.5456 - acc: 0.7255
E

In [13]:
nnmodel_loss, nnmodel_accuracy = nnmodel.evaluate(X_test_minmax, y_test_2, verbose=2)
print(f"Loss: {nnmodel_loss}, Accuracy: {nnmodel_accuracy}")

Loss: 0.4486992735858192, Accuracy: 0.8023089740537968


## Random Forests

### Decide on number of estimators to use

In [14]:
#Load a Random Forest model
model = RandomForestClassifier()

#Set a parameter grid
#Only real parameters we needed to check was the number of estimators
param_grid = {'n_estimators': [20,50,100,200,300]}
grid = GridSearchCV(model, param_grid, verbose=3)

#Fit the grid
grid.fit(X_train_minmax,y_train_sampled)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV] ...................... n_estimators=20, score=0.75, total=   1.2s
[CV] n_estimators=20 .................................................
[CV] ........ n_estimators=20, score=0.7456030150753769, total=   1.0s
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.5s remaining:    0.0s


[CV] ........ n_estimators=20, score=0.7788944723618091, total=   0.9s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7638190954773869, total=   2.8s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7807788944723618, total=   2.4s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7788944723618091, total=   3.1s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7619346733668342, total=   4.7s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7726130653266332, total=   4.1s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7701005025125628, total=   5.3s
[CV] n_estimators=200 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [16]:
#Find the best score and best parameters
print(f'The best score was {grid.score(X_test_minmax,y_test)}\nThe best parameters were {grid.best_params_}')

The best score was 0.8045108307545823
The best parameters were {'n_estimators': 50}


In [17]:
#Take a look at classification report to confirm that the model is ideal
print(classification_report(y_test, grid.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       0.98      0.81      0.89     15780
          1       0.20      0.77      0.32      1024

avg / total       0.93      0.80      0.85     16804



In [18]:
#Take a look at the confusion matrix to confirm that there is no unintended behavior (e.g. only guessing unbroken)
confusion_matrix(y_test, grid.predict(X_test_minmax))

array([[12734,  3046],
       [  239,   785]], dtype=int64)

# Performing deeper analysis with Random Forests

In [19]:
#Apply cross validation so as to have percentage predictions for all data points without bias towards training data
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
split = cv.split(pipes_df3[inputColumns],pipes_df3['FAILURE'])
rfmodel = RandomForestClassifier(n_estimators=300, random_state=42)

In [20]:
#Create new column for use in displaying results of analysis to be filled in later
#Final results will be stored in final_df
scores = []
final_df = pipes_df3
final_df['BurstChance'] = 0
confusion = []

In [21]:
#For each train-test split from our crossvalidation, run the same type of analysis as before
for train_idx, test_idx in cv.split(pipes_df3[inputColumns],pipes_df3['FAILURE']):
    Current_X_train = pipes_df3[inputColumns].loc[train_idx]
    Current_X_test = pipes_df3[inputColumns].loc[test_idx]
    Current_y_train = pipes_df3['FAILURE'].loc[train_idx]
    Current_y_test = pipes_df3['FAILURE'].loc[test_idx]
    
    
    #Undersample training data as before
    Current_X_train_false = Current_X_train[Current_y_train==0].sample(n=sum(Current_y_train))
    Current_X_train_true = Current_X_train[Current_y_train==1].sample(n=sum(Current_y_train))
    
    Current_X_train_sampled = Current_X_train_true.append(Current_X_train_false)
    Current_y_train_sampled = Current_y_train.loc[Current_X_train_sampled.index]
    
    Current_X_minmax = MinMaxScaler().fit(Current_X_train)
    Current_X_train_minmax = Current_X_minmax.transform(Current_X_train_sampled)
    Current_X_test_minmax = Current_X_minmax.transform(Current_X_test)

    
    rfmodel.fit(Current_X_train_minmax, Current_y_train_sampled)
    score = rfmodel.score(Current_X_test_minmax, Current_y_test)
    scores.append(score)
    currentconfusion = confusion_matrix(Current_y_test,rfmodel.predict(Current_X_test_minmax))
    print(currentconfusion)
    confusion.append([currentconfusion[0][0],currentconfusion[0][1],currentconfusion[1][0],currentconfusion[1][1]])
    
    #Store prediction probabilities into final_df
    final_df['BurstChance'].loc[test_idx] = [x[1] for x in list(rfmodel.predict_proba(Current_X_test_minmax))]


[[8425 2095]
 [ 119  564]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8460 2060]
 [ 134  549]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8468 2052]
 [ 129  553]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8328 2192]
 [ 127  555]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8436 2084]
 [ 150  532]]


C:\Users\Amy Perez\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
#Look at combined confusion matrix
truepositive = confusion[0][0] + confusion[1][0]+confusion[2][0]+confusion[3][0]+confusion[4][0]
falsenegative = confusion[0][1] + confusion[1][1]+confusion[2][1]+confusion[3][1]+confusion[4][1]
falsepositive = confusion[0][2] + confusion[1][2]+confusion[2][2]+confusion[3][2]+confusion[4][2]
truenegative = confusion[0][3] + confusion[1][3]+confusion[2][3]+confusion[3][3]+confusion[4][3]


In [23]:
print(f'[{[truepositive, falsenegative]}\n{[falsepositive,truenegative]}]')

[[42117, 10483]
[659, 2753]]


In [24]:
#Look at final scores for each
scores

[0.8023743640096402,
 0.8041596001071142,
 0.8053026245313337,
 0.7929833958221746,
 0.8005713265488306]

In [25]:
#Output top ten most likely pipes to burst which are currently unbroken according to our model
final_df[['BurstChance','FID','MOID'] + inputColumns].loc[final_df['FAILURE']==0].sort_values('BurstChance', ascending=False).head(10)

,BurstChance,FID,MOID,DIAMETER,PIPE_LENGTH,ROUGHNESS,FLOW,VELOCITY,HEADLOSS,HL1000,...,MATERIAL_RCP,MATERIAL_SP,MATERIAL_ST,MATERIAL_T.D,MATERIAL_T.D.,MATERIAL_TTE,MATERIAL_TYT,MATERIAL_UNK,MATERIAL_W.I.,Age
32757,0.966667,12184,P29259,6.0,565.147169,84,40.2327,0.4565,0.2451,0.4337,...,0,0,0,0,0,0,0,0,0,52
29863,0.960000,18278,P52337,6.0,656.275538,84,2.2905,0.0260,0.0013,0.0020,...,0,0,0,0,0,0,0,0,0,60
55116,0.960000,19747,P50077,6.0,847.919534,121,29.4955,0.3347,0.1052,0.1241,...,0,0,0,0,0,0,0,1,0,51
1913,0.960000,10000,P29568,6.0,536.392085,39,11.6671,0.1324,0.0973,0.1814,...,0,0,0,0,0,0,0,0,0,82
53977,0.960000,17100,P51125,6.0,331.339401,39,13.6345,0.1547,0.0802,0.2420,...,0,0,0,0,0,0,0,1,0,86
2018,0.956667,28728,P38796,6.0,425.384105,39,31.5878,0.3584,0.4880,1.1473,...,0,0,0,0,0,0,0,0,0,80
1985,0.956667,9505,P33450,6.0,644.597451,39,15.5969,0.1770,0.2002,0.3106,...,0,0,0,0,0,0,0,0,0,80
39019,0.956667,18254,P52004,6.0,450.160677,121,2.1036,0.0239,0.0004,0.0008,...,0,0,0,0,1,0,0,0,0,46
2080,0.953333,19705,P39470,6.0,554.608068,39,13.0873,0.1485,0.1245,0.2245,...,0,0,0,0,0,0,0,0,0,79
2254,0.953333,17720,P45010,6.0,321.106136,39,22.2302,0.2522,0.1923,0.5987,...,0,0,0,0,0,0,0,0,0,77


In [26]:
#Output most relevant parameters according to our recent model
print(sorted(zip(map(lambda x: round(x, 4), rfmodel.feature_importances_), inputColumns), reverse=True))

[(0.0894, 'PIPE_LENGTH'), (0.0794, 'Age'), (0.0414, 'MAX_HEAD'), (0.0377, 'MIN_HEAD'), (0.0366, 'DEMAND'), (0.0357, 'MAX_HEADLOSS'), (0.0357, 'HEAD'), (0.0353, 'AVE_HEAD'), (0.035, 'AVE_HEADLOSS'), (0.0334, 'ROUGHNESS'), (0.0322, 'DIFF_PRESS'), (0.029, 'HEADLOSS'), (0.0288, 'MIN_PRESS'), (0.0286, 'MAX_FLOW'), (0.0282, 'FLOW'), (0.0276, 'MAX_VELOCITY'), (0.0275, 'MIN_FLOW'), (0.0275, 'AVE_FLOW'), (0.0269, 'DIAMETER'), (0.0267, 'ELEVATION'), (0.0267, 'AVE_PRESS'), (0.0264, 'RUN_ELEV'), (0.0261, 'AVE_VELOCITY'), (0.026, 'MIN_VELOCITY'), (0.0254, 'MAX_PRESS'), (0.0252, 'VELOCITY'), (0.0248, 'PRESSURE'), (0.0246, 'HL1000'), (0.0239, 'MIN_HEADLOSS'), (0.0069, 'MATERIAL_T.D.'), (0.0068, 'MATERIAL_M.J.'), (0.0049, 'MATERIAL_DIP'), (0.0032, 'MATERIAL_TYT'), (0.0017, 'MATERIAL_L.J.'), (0.0017, 'MATERIAL_DEL'), (0.001, 'MATERIAL_UNK'), (0.0006, 'MATERIAL_W.I.'), (0.0005, 'MATERIAL_CPP'), (0.0003, 'MATERIAL_CO'), (0.0002, 'MATERIAL_ST'), (0.0002, 'MATERIAL_OTH'), (0.0002, 'MATERIAL_CI'), (0.0001, 

In [27]:
len(inputColumns)

57

In [28]:
print(classification_report(y_test,rfmodel.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       1.00      0.81      0.89     15780
          1       0.24      0.94      0.39      1024

avg / total       0.95      0.82      0.86     16804



In [29]:
final_df.to_csv('output2.csv')